In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping

# загружаем изображения MNIST из модуля datasets
(x_train, _),(x_test, _) = mnist.load_data()

#конвертируем набор данных в диапазон от 0 до 1
x_train = x_train.astype('float32')/255
x_train = np.reshape(x_train,(len(x_train),28,28,1))
x_test = x_test.astype('float32')/255
x_test = np.reshape(x_test,(len(x_test),28,28,1))

#добавляем гауссовский шум на изображения, зашумляя их
noise_factor = 0.1 # 0.1 0.3 0.5 0.7 0.9
x_train_noisy = x_train + noise_factor * np.random.normal(0, 1, size = x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(0, 1, size = x_test.shape)
x_train_noisy = np.clip(x_train_noisy, 0, 1)
x_test_noisy = np.clip(x_test_noisy, 0, 1)

# глубокий свёрточный автокодировщик
def create_deep_conv_ae():
  # определяем входящий слой и создаём модель нейронной сети
  #кодирование
  input_img = Input(shape=(28,28,1)) # 28, 28, 1 - размерности строк, столбцов, фильтров
одной картинки, без батч-размерности

  x = Conv2D(128,(7,7),activation='relu',padding='same')(input_img)
  x = MaxPooling2D((2,2),padding='same')(x)
  x = Conv2D(32,(2,2),activation='relu',padding='same')(x)
  x = MaxPooling2D((2,2),padding='same')(x)
  encoded = Conv2D(1,(7,7),activation='relu',padding='same')(x)

  # «бутылочное горлышко» закодировано и состоит из сжатых изображений
  # расшифровка
  input_encoded = Input(shape=(7,7,1)) # представление 49-размерное

  x = Conv2D(32,(7, 7),activation='relu',padding='same')(input_encoded)
  x = UpSampling2D((2,2))(x)
  x = Conv2D(128,(2,2),activation='relu',padding='same')(x)
  x = UpSampling2D((2,2))(x)
  decoded = Conv2D(1,(7,7),activation='sigmoid',padding='same')(x)

  # обучаем (тренируем) автокодировщик
  encoder = Model(input_img, encoded, name="encoder")
  encoder.summary()
  decoder = Model(input_encoded, decoded, name="decoder")
  decoder.summary()
  autoencoder = Model(input_img, decoder(encoder(input_img)), name="autoencoder")
  autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
  autoencoder.summary()
  return encoder, decoder, autoencoder

# свёрточный автокодировщик, маленькое "бутылочное горлышко"
def create_conv_ae():
  # определяем входящий слой и создаём модель нейронной сети
  #кодирование
  input_img = Input(shape=(28,28,1)) # 28, 28, 1 - размерности строк, столбцов, фильтров
одной картинки, без батч-размерности

  x = Conv2D(16,(7,7),activation='relu',padding='same')(input_img)
  x = MaxPooling2D((2,2),padding='same')(x)
  x = Conv2D(1,(2,2),activation='relu',padding='same')(x)
  x = MaxPooling2D((2,2),padding='same')(x)
  encoded = Conv2D(1,(7,7),activation='relu',padding='same')(x)

  # «бутылочное горлышко» закодировано и состоит из сжатых изображений
  # расшифровка
  input_encoded = Input(shape=(7,7,1)) # представление 49-размерное

  x = Conv2D(1,(7, 7),activation='relu',padding='same')(input_encoded)
  x = UpSampling2D((2,2))(x)
  x = Conv2D(16,(2,2),activation='relu',padding='same')(x)
  x = UpSampling2D((2,2))(x)
  decoded = Conv2D(1,(7,7),activation='sigmoid',padding='same')(x)

  # обучаем (тренируем) автокодировщик
  encoder = Model(input_img, encoded, name="encoder")
  encoder.summary()
  decoder = Model(input_encoded, decoded, name="decoder")
  decoder.summary()
  autoencoder = Model(input_img, decoder(encoder(input_img)), name="autoencoder")
  autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
  autoencoder.summary()
  return encoder, decoder, autoencoder

# сжимающий автокодировщик
def create_dense_ae():
  # определяем входящий слой и создаём модель нейронной сети
  #кодирование
  input_img = Input(shape=(28,28,1)) # 28, 28, 1 - размерности строк, столбцов, фильтров
одной картинки, без батч-размерности
  # вспомогательный слой решейпинга
  flat_img = Flatten()(input_img)

  # кодированное полносвязным слоем представление
  encoded = Dense(49,activation='relu')(flat_img)

  # расшифровка
  # раскодированное другим полносвязным слоем изображение
  input_encoded = Input(shape=(49,))
  flat_decoded = Dense(28*28,activation='sigmoid')(input_encoded)
  decoded = Reshape((28,28,1))(flat_decoded)

  # обучаем (тренируем) автокодировщик
  encoder = Model(input_img, encoded, name="encoder")
  encoder.summary()
  decoder = Model(input_encoded, decoded, name="decoder")
  decoder.summary()
  autoencoder = Model(input_img, decoder(encoder(input_img)), name="autoencoder")
  autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
  autoencoder.summary()
  return encoder, decoder, autoencoder

# глубокий сжимающий автокодировщик
def create_deep_dense_ae():
  # определяем входящий слой и создаём модель нейронной сети
  #кодирование
  input_img = Input(shape=(28,28,1)) # 28, 28, 1 - размерности строк, столбцов, фильтров
одной картинки, без батч-размерности
  # вспомогательный слой решейпинга
  flat_img = Flatten()(input_img)

  # кодированное полносвязным слоем представление
  x = Dense(49*3,activation='relu')(flat_img)
  x = Dense(49*2,activation='relu')(x)
  encoded = Dense(49,activation='linear')(flat_img)
  encoded = Dense(49, activation='linear')(x)

  # расшифровка
  # раскодированное другим полносвязным слоем изображение
  input_encoded = Input(shape=(49,))
  x = Dense(49*2, activation='relu')(input_encoded)
  x = Dense(49*3, activation='relu')(x)
  flat_decoded = Dense(28*28, activation='sigmoid')(x)
  decoded = Reshape((28,28,1))(flat_decoded)

  # обучаем (тренируем) автокодировщик
  encoder = Model(input_img, encoded, name="encoder")
  encoder.summary()
  decoder = Model(input_encoded, decoded, name="decoder")
  decoder.summary()
  autoencoder = Model(input_img, decoder(encoder(input_img)), name="autoencoder")
  autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
  autoencoder.summary()
  return encoder, decoder, autoencoder

# автокодирование
#encoder, decoder, autoencoder = create_deep_conv_ae()
#encoder, decoder, autoencoder = create_conv_ae()
#encoder, decoder, autoencoder = create_dense_ae()
encoder, decoder, autoencoder = create_deep_dense_ae()
autoencoder.fit(x_train_noisy, x_train, epochs=5, batch_size=256, shuffle=True,
validation_data=(x_test, x_test))
#autoencoder.fit(x_train_noisy, x_train, validation_split=0.2, epochs=5, batch_size=256,
#shuffle=True)

#получаем предсказания по зашумлённым данным
x_test_result = autoencoder.predict(x_test_noisy)

# количество изображений
n = 10

#визуализируем оригинальные изображения
print('\nОригинальные изображения:')
plt.figure(figsize=(12,2))
for i in range(n):
  ax = plt.subplot(1, n, i+1)
  plt.imshow(x_test[i].reshape(28,28))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

plt.show()

#визуализируем зашумлённые изображения
print('\nЗашумлённые изображения:')
plt.figure(figsize=(12,2))
for i in range(n):
  ax=plt.subplot(1,n,i+1)
  plt.imshow(x_test_noisy[i].reshape(28,28))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

plt.show()

#заново визуализируем воссозданные изображения
print('\nВоссозданные изображения:')
plt.figure(figsize=(12,2))
for i in range(n):
  ax=plt.subplot(1,n,i+1)
  plt.imshow(x_test_result[i].reshape(28,28))
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

plt.show()